In [1]:
# here idea is to use pyspark for clustering the data of seed_dataset

# first we have used jupyter notebook at local machine to import data from github then added column names and then
#remove class from the dataset. after saved the dataset as csv file. that file is now uploaded to the datbricks and clustering is prerformed using pyspark. 
from pyspark.sql import SparkSession

In [2]:
# loading dataset as dataframe
df = spark.read.csv('/FileStore/tables/seeds_dataset.csv', inferSchema=True, header=True)
df.show(3) # in spark we use show or take and argument will be number of element to be passed.

+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient|length_of_groove|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|
14.88| 14.57| 0.8811|5.553999999999999| 3.333| 1.018| 4.956|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+
only showing top 3 rows

In [3]:
df.printSchema() # printing schema of the dataset

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [4]:
from pyspark.ml.feature import VectorAssembler # in spark api we have to convert data to vectors in order to run the model.
# kmeans is imported from pyspark.ml.clustering
from pyspark.ml.clustering import KMeans

# creating an instance of the vector assembler 
assembler = VectorAssembler(inputCols = df.columns, outputCol = 'features')

# transforming dataframe into vector assembler 
final_df = assembler.transform(df)
final_df.show(3)

+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient|length_of_groove| features|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|
14.88| 14.57| 0.8811|5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+
only showing top 3 rows

In [5]:
final_df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [6]:
# in clustering we have to scale the features so as to reduce the distance and which helps in computation become faster
from pyspark.ml.feature import StandardScaler

# created instance of the standard scaler
scaler = StandardScaler(inputCol = 'features', outputCol = 'scaledFeatures')

#fitting the vector data and transforming with scaler transformation
scaler_model = scaler.fit(final_df)
final_df = scaler_model.transform(final_df)
final_df.show(3)

+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+--------------------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient|length_of_groove| features| scaledFeatures|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+--------------------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...|
14.88| 14.57| 0.8811|5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...|
+-----+---------+-----------+-----------------+------------------+---------------------+----------------+--------------------+--------------------+
only showing top 3 rows

In [7]:
# first row of the final scaled vecorized dataset
final_df.take(1)

Out[ 7 ]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [8]:

#instantiated kmeans with 3 cluster 
kmeans = KMeans(featuresCol = 'scaledFeatures', k=3)

# fitting the model
model = kmeans.fit(final_df)

In [9]:
# sum of square error within cluster

#sum of squared  error within cluster is used to get the right number of clusters for the model. we use elbow method in which we plot WSSSE on y axis and # number of cluster on x axis . as we move to more number of cluster error reduced but where we see that curve is started changing gradually that point we # take as optimum number of cluster. for more info check Elbow method.
print('WSSSE:', model.computeCost(final_df))

('WSSSE:', 429.07558670547337)

In [10]:
# cluster centeroids
centers = model.clusterCenters()
print(centers)

[array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,
 9.748067 , 2.39850262, 12.2661748 ]), array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,
 8.55443412, 1.81649411, 10.32998598]), array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,
 7.50395937, 3.27185132, 10.42126018])]

In [11]:
# prdicting the data. transform method is same as sklearn model.predict. it gives many output but we are printing scaledfeatures and prediction
model.transform(final_df).select('scaledFeatures', 'prediction').show()

+--------------------+----------+
 scaledFeatures|prediction|
+--------------------+----------+
[5.24452795332028...| 1|
[5.11393027165175...| 1|
[4.91116018695588...| 1|
[4.75650503761158...| 1|
[5.54696468981581...| 1|
[4.94209121682475...| 1|
[5.04863143081749...| 1|
[4.84929812721816...| 1|
[5.71536696354628...| 0|
[5.65006812271202...| 0|
[5.24452795332028...| 1|
[4.82180387844584...| 1|
[4.77368894309428...| 1|
[4.73588435103234...| 1|
[4.72213722664617...| 1|
[5.01426361985209...| 1|
[4.80805675405968...| 1|
[5.39230954047151...| 1|
[5.05206821191403...| 1|
[4.37158555479908...| 2|
+--------------------+----------+
only showing top 20 rows

In [12]:
model.transform(final_df).show() # all from transfrom method

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
 area|perimeter|compactness| length_of_kernel| width_of_kernel|asymmetry_coefficient| length_of_groove| features| scaledFeatures|prediction|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
15.26| 14.84| 0.871| 5.763| 3.312| 2.221| 5.22|[15.26,14.84,0.87...|[5.24452795332028...| 1|
14.88| 14.57| 0.8811| 5.553999999999999| 3.333| 1.018| 4.956|[14.88,14.57,0.88...|[5.11393027165175...| 1|
14.29| 14.09| 0.905| 5.291|3.3369999999999997| 2.699| 4.825|[14.29,14.09,0.90...|[4.91116018695588...| 1|
13.84| 13.94| 0.8955| 5.324|3.3789999999999996| 2.259| 4.805|[13.84,13.94,0.89...|[4.75650503761158...| 1|
16.14| 14.99| 0.9034|5.6579999999999995| 3.562| 1.355| 5.175|[16.14,14.99,0.90...|[5.54696468981581...| 1|
14.38| 14.21| 0.8951| 5.386| 3.312| 2.4619999999999997| 4.956|[14.38,14.21,0.89...|[4.94209121682475...| 1|
14.69| 14.49| 0.8799| 5.563| 3.259| 3.5860000000000003| 5.218999999999999|[14.69,14.49,0.87...|[5.04863143081749...| 1|
14.11| 14.1| 0.8911| 5.42| 3.302| 2.7| 5.0|[14.11,14.1,0.891...|[4.84929812721816...| 1|
16.63| 15.46| 0.8747| 6.053| 3.465| 2.04| 5.877000000000001|[16.63,15.46,0.87...|[5.71536696354628...| 0|
16.44| 15.25| 0.888|5.8839999999999995| 3.505| 1.969|5.5329999999999995|[16.44,15.25,0.88...|[5.65006812271202...| 0|
15.26| 14.85| 0.8696|5.7139999999999995| 3.242| 4.543| 5.314|[15.26,14.85,0.86...|[5.24452795332028...| 1|
14.03| 14.16| 0.8796| 5.438| 3.201| 1.7169999999999999| 5.001|[14.03,14.16,0.87...|[4.82180387844584...| 1|
13.89| 14.02| 0.888| 5.439| 3.199| 3.986| 4.738|[13.89,14.02,0.88...|[4.77368894309428...| 1|
13.78| 14.06| 0.8759| 5.479| 3.156| 3.136| 4.872|[13.78,14.06,0.87...|[4.73588435103234...| 1|
13.74| 14.05| 0.8744| 5.482| 3.114| 2.932| 4.825|[13.74,14.05,0.87...|[4.72213722664617...| 1|
14.59| 14.28| 0.8993| 5.351| 3.333| 4.185| 4.781000000000001|[14.59,14.28,0.89...|[5.01426361985209...| 1|
13.99| 13.83| 0.9183| 5.119| 3.383| 5.234| 4.781000000000001|[13.99,13.83,0.91...|[4.80805675405968...| 1|
15.69| 14.75| 0.9058| 5.527| 3.514| 1.599| 5.046|[15.69,14.75,0.90...|[5.39230954047151...| 1|
 14.7| 14.21| 0.9153| 5.205| 3.466| 1.767| 4.649|[14.7,14.21,0.915...|[5.05206821191403...| 1|
12.72| 13.57| 0.8686| 5.226| 3.049| 4.102| 4.914|[12.72,13.57,0.86...|[4.37158555479908...| 2|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+----------+
only showing top 20 rows